In [30]:
import numpy as np
import pandas as pd
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import multi_gpu_model
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [31]:
def split_data(X, y, test_data_size):
    '''
    Split data into test and training datasets.

    INPUT
        X: NumPy array of arrays
        y: Pandas series, which are the labels for input array X
        test_data_size: size of test/train split. Value from 0 to 1

    OUPUT
        Four arrays: X_train, X_test, y_train, and y_test
    '''
    return train_test_split(X, y, test_size=test_data_size, random_state=42)


def reshape_data(arr, img_rows, img_cols, channels):
    '''
    Reshapes the data into format for CNN.

    INPUT
        arr: Array of NumPy arrays.
        img_rows: Image height
        img_cols: Image width
        channels: Specify if the image is grayscale (1) or RGB (3)

    OUTPUT
        Reshaped array of NumPy arrays.
    '''
    return arr.reshape(arr.shape[0], img_rows, img_cols, channels)

def cnn_model(train_gen, train_steps, val_gen, val_steps, kernel_size, nb_filters, channels, nb_epoch, batch_size, nb_classes, nb_gpus):
    '''
    Define and run the Convolutional Neural Network

    INPUT
        X_train: Array of NumPy arrays
        X_test: Array of NumPy arrays
        y_train: Array of labels
        y_test: Array of labels
        kernel_size: Initial size of kernel
        nb_filters: Initial number of filters
        channels: Specify if the image is grayscale (1) or RGB (3)
        nb_epoch: Number of epochs
        batch_size: Batch size for the model
        nb_classes: Number of classes for classification

    OUTPUT
        Fitted CNN model
    '''

    model = Sequential()

    '''
    First set of three layers
    Image size: 256 x 256
    nb_filters = 32
    kernel_size = (2,2)
    '''

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                     padding='valid',
                     strides=1,
                     input_shape=(img_rows, img_cols, channels)))
    model.add(Activation('relu'))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    '''
    Second set of three layers
    Image Size: 128 x 128
    nb_filters = 64
    kernel_size = 4,4
    '''

    nb_filters = 64
    kernel_size = (4, 4)

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    # model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    # model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    '''
    Third set of three layers
    Image Size: 64 x 64
    nb_filters = 128
    kernel_size = 8,8
    '''

    nb_filters = 128
    kernel_size = (8, 8)

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))

    # model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
    # model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(12, 12)))

    model.add(Flatten())
    print("Model flattened out to: ", model.output_shape)

    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(4096))
    model.add(Activation("relu"))
    model.add(Dropout(0.2))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    #model = multi_gpu_model(model, gpus=nb_gpus)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print(model.summary())

    
    model_name = 'ChestX'
    checkpoint = ModelCheckpoint(model_name+'/'+model_name+'.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto')
    tensorboard = TensorBoard(log_dir=model_name, batch_size=batch_size, write_graph=True, write_images=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
    earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='auto')
    cvslogger = keras.callbacks.CSVLogger(model_name+'/'+model_name+'.csv', separator=',', append=True)

    callbacks = [checkpoint, reduce_lr, cvslogger, tensorboard, earlystop]
    
    #stop = EarlyStopping(monitor='acc',
                         #min_delta=0.001,atience=2,
                         #verbose=0,
                         #mode='auto')

    #tensor_board = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
    #checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)
    #model.fit_generator(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
     #         verbose=1,
      #        validation_split=0.2,
       #       class_weight='auto',
       #       callbacks=[EarlyStopping(monitor='val_loss',patience=3),checkpointer])
    model.fit_generator(train_gen, train_steps, epochs=nb_epoch, verbose=1, 
                    max_queue_size=2, validation_data=val_gen, 
                    validation_steps=val_steps, shuffle=True,
                    workers=1, use_multiprocessing= False, 
                    initial_epoch=0, callbacks=callbacks)
    
    return model  
#callbacks=[stop, tensor_board]

In [32]:
def batch_generator(data_X, data_y, batch_size):
    indexes = np.array(range(len(data_y)))
    n = len(indexes)
    while True:
        batch_start = 0
        batch_end = batch_size
        np.random.shuffle(indexes)
        while batch_start < n:
            index = []
            batch_y = []
            y = []
            index = indexes[batch_start:batch_end]
            batch_x = np.array([data_X[i] for i in index])
            batch_y = np.array([data_y[i] for i in index])
            yield batch_x, batch_y
            batch_start += batch_size   
            batch_end += batch_size

In [33]:
if __name__ == '__main__':

    batch_size = 100
    nb_classes = 2
    nb_epoch = 2
    nb_gpus = 8
    img_rows, img_cols = 256,256
    channels = 1
    nb_filters = 32
    kernel_size = (2, 2)

    # Import data
    labels = pd.read_csv("sample_labels.csv")
    X = np.load("X_sample.npy")
    y = labels.Finding_Labels
    # y = np.array(pd.get_dummies(y))
    #label_encoder = LabelEncoder()
    #y = label_encoder.fit_transform(y)
    y = y.reshape(-1, 1)


In [34]:
    print(X.shape)
    print(y.shape)

(96084, 256, 256)
(96084, 1)


In [35]:
    print("Splitting data into train/val/test datasets")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)
    




Splitting data into train/val/test datasets
(61493, 256, 256)
(61493, 1)
(19217, 256, 256)
(19217, 1)


In [36]:
    print("Reshaping Data")
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, channels)
    X_val = X_val.reshape(X_val.shape[0],img_rows, img_cols, channels)
    print("X_train Shape: ", X_train.shape)
    print("X_test Shape: ", X_test.shape)
    print("X_val Shape:", X_val.shape)

    input_shape = (img_rows, img_cols, channels)

Reshaping Data
('X_train Shape: ', (61493, 256, 256, 1))
('X_test Shape: ', (19217, 256, 256, 1))
('X_val Shape:', (15374, 256, 256, 1))


In [37]:
    #X_train = X_train[1:70000]
    #y_train = y_train[1:70000]
    #X_test = X_test[1:10000]
    #y_test = y_test[1:10000]
    #print(X_train.shape)


In [38]:
    #print("Normalizing Data")
    #X_train = X_train.astype(np.float32)
    #X_test = X_test.astype(np.float32)
    #X_val = X_val.astype(np.float32)
    #X_train = X_train/255
    #X_test = X_test/255

    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    y_val = np_utils.to_categorical(y_val, nb_classes)
    print("y_train Shape: ", y_train.shape)
    print("y_test Shape: ", y_test.shape)

IndexError: index 10 is out of bounds for axis 1 with size 2

In [ ]:
    train_gen = batch_generator(X_train, y_train, batch_size)
    val_gen = batch_generator(X_val, y_val, batch_size)
    test_gen = batch_generator(X_test, y_test, batch_size)

In [ ]:
    train_steps = int(len(y_train)//batch_size)
    val_steps = int(len(y_test)//batch_size)

    #model = cnn_model(X_train, y_train, kernel_size, nb_filters, channels, nb_epoch, batch_size, nb_classes, nb_gpus)
    model = cnn_model(train_gen, train_steps,val_gen,val_steps, kernel_size, nb_filters, channels, nb_epoch, batch_size, nb_classes, nb_gpus)

    print("Predicting")
    X_test, y_test = test_gen.next()
    y_pred = model.predict(X_test)

    y_test = np.argmax(y_test, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average="weighted")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)

In [ ]:
    print("Predicting")
    X_test, y_test = test_gen.next()
    y_pred = model.predict(X_test)

    y_test = np.argmax(y_test, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average="weighted")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)